# helper

> Helper functions 

In [ ]:
#| default_exp helper.__init__

In [ ]:
#| export
from __future__ import annotations



In [ ]:
import operator


from fastcore.test import *

In [ ]:
#| export
def substring_generator(input_string: str):
    length = len(input_string)
    for i in range(length):
        for j in range(i + 1, length + 1):
            yield input_string[i:j]


def sublist_generator(input_list: list):
    length = len(input_list)
    for i in range(length):
        for j in range(i + 1, length + 1):
            yield input_list[i:j]